In [14]:
#importing dependencies and setting app token
import pandas as pd
import requests
from datetime import date, datetime
from sodapy import Socrata
from api_keys import MyAppToken
#MyAppToken = 'R6znHRxdUAlL3CBi0ylsw6dCP'

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
day = today.strftime("%d")
print("Day =", day)

Day = 14


In [17]:
#still need a way to filter 
client = Socrata("data.cityofchicago.org", MyAppToken, None, None )

#getting () results from the soda api
try:
    results = client.get(
        "ijzp-q8t2"
        , limit=20
        , exclude_system_fields=True
    )
                         
except requests.exceptions.Timeout:
  print("Timeout occurred")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

Timeout occurred


,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_d3ds_rm58,:@computed_region_d9mm_jgwp,:@computed_region_rpca_8um6,:@computed_region_vrxf_vc4k,arrest,beat,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,5,21853,3,357,32,11,9,10,False,2422,...,42.018734342,"{'latitude': '42.018734342', 'longitude': '-87...",GAS STATION,-87.66588196,THEFT,2020-04-13T15:55:11.000,49,1165619,1950124,2020
1,2,22257,19,297,266,17,23,65,False,0713,...,41.790019182,"{'latitude': '41.790019182', 'longitude': '-87...",STREET,-87.65841048,BATTERY,2020-04-13T15:55:11.000,16,1168305,1866796,2020
2,4,21559,53,403,268,24,60,7,False,0225,...,41.791021009,"{'latitude': '41.791021009', 'longitude': '-87...",SIDEWALK,-87.625158623,BATTERY,2020-04-13T15:55:11.000,20,1177370,1867234,2020
3,38,21849,28,120,48,1,20,14,False,1712,...,41.969349668,"{'latitude': '41.969349668', 'longitude': '-87...",STREET,-87.725634004,THEFT,2020-04-13T15:55:11.000,39,1149510,1932007,2020
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,0621,...,NaN,NaN,PARKING LOT / GARAGE (NON RESIDENTIAL),NaN,MOTOR VEHICLE THEFT,2020-04-13T15:55:11.000,17,NaN,NaN,2020


In [10]:
#dropping columns that have no use
clean_columns_df = results_df.drop(columns=['location','x_coordinate', 'y_coordinate','id','beat','district','ward','community_area','fbi_code','updated_on', 'case_number', 'block', 'iucr', 'location_description', 'arrest', ':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i', ':@computed_region_43wa_7qmu' , ':@computed_region_rpca_8um6',':@computed_region_d9mm_jgwp',':@computed_region_d3ds_rm58'])
#adding chicago column for organization while grouping
clean_columns_df['city'] = "Chicago"
#formatting datetime to ease readability
clean_columns_df['month'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%m')
clean_columns_df['date'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%d')
clean_columns_df.rename(columns = {'date':'day'}, inplace = True)

clean_columns_df.head()

,day,description,domestic,latitude,longitude,primary_type,year,city,month
0,06,$500 AND UNDER,False,42.018734342,-87.66588196,THEFT,2020,Chicago,04
1,06,DOMESTIC BATTERY SIMPLE,True,41.790019182,-87.65841048,BATTERY,2020,Chicago,04
2,06,AGGRAVATED - HANDGUN,False,41.791021009,-87.625158623,BATTERY,2020,Chicago,04
3,06,OVER $500,False,41.969349668,-87.725634004,THEFT,2020,Chicago,04
4,06,AUTOMOBILE,False,NaN,NaN,MOTOR VEHICLE THEFT,2020,Chicago,04


In [6]:
#reorganizing columns to read easier left to right
clean_columns_df = clean_columns_df[["city",'primary_type','description','domestic', "day", "month", "year", "latitude", "longitude"]]

In [14]:
domestic_crimes = clean_columns_df['domestic'] == True
non_domestic_crimes = clean_columns_df['domestic'] == False
domestic_crimes_df = clean_columns_df[domestic_crimes]
non_domestic_crimes_df = clean_columns_df[non_domestic_crimes]

domestic_crimes_df

,city,primary_type,description,domestic,day,month,year,latitude,longitude
0,Chicago,CRIMINAL DAMAGE,TO PROPERTY,True,02,04,2020,41.699186904,-87.614767977
1,Chicago,ASSAULT,SIMPLE,True,02,04,2020,41.882254983,-87.763964363
2,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,02,04,2020,41.88625035,-87.75895272
4,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,02,04,2020,41.855797367,-87.679409498
7,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,02,04,2020,41.979498742,-87.839052959
...,...,...,...,...,...,...,...,...,...
19952,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,24,02,2020,41.69189982,-87.642367883
19969,Chicago,OTHER OFFENSE,TELEPHONE THREAT,True,24,02,2020,41.747629731,-87.657300893
19978,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,24,02,2020,41.837582116,-87.729230812
19984,Chicago,BATTERY,DOMESTIC BATTERY SIMPLE,True,24,02,2020,41.876808122,-87.686326186


In [15]:
#still have to use day to filter results from last year
non_domestic_crimes_df

,city,primary_type,description,domestic,day,month,year,latitude,longitude
3,Chicago,ROBBERY,STRONG ARM - NO WEAPON,False,02,04,2020,41.967763746,-87.661181751
5,Chicago,THEFT,$500 AND UNDER,False,02,04,2020,41.880792865,-87.679077595
6,Chicago,ROBBERY,STRONG ARM - NO WEAPON,False,02,04,2020,41.884498505,-87.741458652
9,Chicago,BATTERY,SIMPLE,False,02,04,2020,41.886811027,-87.755293578
11,Chicago,HOMICIDE,FIRST DEGREE MURDER,False,02,04,2020,41.872820671,-87.701596513
...,...,...,...,...,...,...,...,...,...
19994,Chicago,ASSAULT,SIMPLE,False,24,02,2020,41.9315849,-87.742677317
19995,Chicago,THEFT,$500 AND UNDER,False,24,02,2020,41.892507682,-87.626238686
19996,Chicago,CRIMINAL TRESPASS,TO LAND,False,24,02,2020,41.873907046,-87.725430093
19998,Chicago,BURGLARY,FORCIBLE ENTRY,False,24,02,2020,41.812641085,-87.639200946


In [16]:
#making dictionaries for mongoDB
non_domestic_chicago_data = non_domestic_crimes_df.to_dict()
domestic_chicago_data = domestic_crimes_df.to_dict()  

In [17]:
non_domestic_chicago_data

{'city': {3: 'Chicago',
  5: 'Chicago',
  6: 'Chicago',
  9: 'Chicago',
  11: 'Chicago',
  12: 'Chicago',
  13: 'Chicago',
  14: 'Chicago',
  15: 'Chicago',
  17: 'Chicago',
  18: 'Chicago',
  19: 'Chicago',
  22: 'Chicago',
  23: 'Chicago',
  24: 'Chicago',
  25: 'Chicago',
  27: 'Chicago',
  28: 'Chicago',
  29: 'Chicago',
  31: 'Chicago',
  32: 'Chicago',
  33: 'Chicago',
  34: 'Chicago',
  35: 'Chicago',
  37: 'Chicago',
  38: 'Chicago',
  39: 'Chicago',
  41: 'Chicago',
  42: 'Chicago',
  43: 'Chicago',
  44: 'Chicago',
  45: 'Chicago',
  46: 'Chicago',
  47: 'Chicago',
  48: 'Chicago',
  49: 'Chicago',
  50: 'Chicago',
  52: 'Chicago',
  53: 'Chicago',
  54: 'Chicago',
  56: 'Chicago',
  57: 'Chicago',
  58: 'Chicago',
  59: 'Chicago',
  60: 'Chicago',
  61: 'Chicago',
  62: 'Chicago',
  63: 'Chicago',
  64: 'Chicago',
  66: 'Chicago',
  68: 'Chicago',
  69: 'Chicago',
  70: 'Chicago',
  71: 'Chicago',
  73: 'Chicago',
  75: 'Chicago',
  77: 'Chicago',
  79: 'Chicago',
  81: 'Chi